In [1]:
import pandas as pd
pd.options.display.max_rows = 100
pd.options.display.max_columns = 50

In [1]:
df = pd.read_pickle("./df_hist.pkl")

In [ ]:
# age restrioction
df['AgeRestriction'] = df['AgeRestriction'].str.replace(r'(yo|\+)', '', regex=True)
df['AgeRestriction'] = df['AgeRestriction'].str.replace("&", "-")
df['AgeRestriction'] = df['AgeRestriction'].str.replace("Pour ", "")
possible_ages = list(map(str, range(2, 11)))

# Function to set age variables based on 'x' values
def set_age_variables(AgeRestriction):
    age_variables = ['age' + age for age in possible_ages]
    if '-' in AgeRestriction:
        range_start, range_end = map(int, AgeRestriction.split('-'))
        age_values = [1 if range_start <= int(age) <= range_end else 0 for age in possible_ages]
        return pd.Series(age_values, index=age_variables)
    else:
        return pd.Series([1 if AgeRestriction == age else 0 for age in possible_ages], index=age_variables)

# Apply the function to create age variables
df[possible_ages] = df['AgeRestriction'].apply(set_age_variables)


possible_ages = list(map(str, range(2, 11)))

df[possible_ages] = df['AgeRestriction'].apply(set_age_variables)

for i in range(2, 11):
    df.rename(columns={str(i): f'PossibleAge{i}'}, inplace=True)


# distance
df['distance_qcut'] = pd.qcut(df['Distance'], q=10, labels=False, precision=0)

# course_indicator
df['course_indicator_g'] = df['CourseIndicator'] == "G"
df['course_indicator_p'] = df['CourseIndicator'] == "P"
df['course_indicator_&'] = df['CourseIndicator'] == "&"

df.drop(columns=['CourseIndicator'], inplace=True)

# race condition
df['class_restirction_maxwonprize'] = df['ClassRestriction'].str.extract(r'NW\$(\d+)', expand=False)
df['class_restirction_maxwonprize'] = pd.qcut(pd.to_numeric(df['class_restirction_maxwonprize']), q=10, labels=False)
df['class_restirction_racecondition'] = df['ClassRestriction'].str.extract(r'NW\$\d+\s?(\w{2,3})?$', expand=False)
df= pd.get_dummies(df, columns=["class_restirction_racecondition"], prefix="class_restirction_racecondition")
df['class_restirction_maxwonprize'][df['class_restirction_maxwonprize'].isna()] = 10

# foaling country
df= pd.get_dummies(df, columns=["FoalingCountry"], prefix="country")

# shoes
df= pd.get_dummies(df, columns=["FrontShoes"], prefix="front_shoes")
df= pd.get_dummies(df, columns=["HindShoes"], prefix="hind_shoes")

# gender
df['gender'] = df['Gender'].map({'M': 0, 'F': 1})
df.drop(columns=['Gender'], inplace=True)

# going
df= pd.get_dummies(df, columns=["GoingAbbrev"], prefix="going_abbrev")

# handicap
df= pd.get_dummies(df, columns=["HandicapType"], prefix="handicap_type")

# race group
df= pd.get_dummies(df, columns=["RaceGroup"], prefix="race_group")

# race prize
df['race_prize_money_qcut'] = pd.qcut(df['RacePrizemoney'], q=10, labels=False, precision=0)

# race time of day, season
df['RaceStartTime'] = pd.to_datetime(df['RaceStartTime'])
df['Hour'] = df['RaceStartTime'].dt.hour

def get_season(date):
    if date.month in (3, 4, 5):
        return 'spring'
    elif date.month in (6, 7, 8):
        return 'summer'
    elif date.month in (9, 10, 11):
        return 'fall'
    else:
        return 'winter'

def assign_time_of_day(hour):
    if 10 <= hour < 15:
        return 'midDay'
    elif 15 <= hour < 19:
        return 'evening'
    elif 19 <= hour <= 23:
        return 'night'
    else:
        return 'unknown'

# Apply the function to create the new variable
df['time_of_day'] = df['Hour'].apply(assign_time_of_day)
df['season'] = df['RaceStartTime'].apply(get_season)

df = df.drop("Hour", axis=1)

df= pd.get_dummies(df, columns=["time_of_day"], prefix="time_of_day")
df= pd.get_dummies(df, columns=["season"], prefix="season")

df= pd.get_dummies(df, columns=["RacingSubType"], prefix="racing_sub_type")
df= pd.get_dummies(df, columns=["Barrier"], prefix="barrier")
df= pd.get_dummies(df, columns=["SexRestriction"], prefix="sex_restriction")
df= pd.get_dummies(df, columns=["StartType"], prefix="start_type")
df= pd.get_dummies(df, columns=["Surface"], prefix="surface")
df= pd.get_dummies(df, columns=["Saddlecloth"], prefix="saddle_cloth")
df= pd.get_dummies(df, columns=["StartingLine"], prefix="starting_line")
df= pd.get_dummies(df, columns=["WetnessScale"], prefix="wetness_scale")

# target 
df['TotalPrize'] = df.groupby('RaceID')['Prizemoney'].transform('sum')
df["target"]=df["Prizemoney"]/df["TotalPrize"]

df.rename(columns={"HorseAge":"horse_age", "HandicapDistance": "handicap_distance", "WeightCarried": "weight_carried"}, inplace=True)

In [31]:
filter_columns = [
    "AgeRestriction",
    "ClassRestriction",
    "DamID",
    "Disqualified",
    "Distance",
    "FoalingDate",
    "GoingID",
    "RaceOverallTime",
    "RacePrizemoney",
    "SireID",
    "TrackID",
    "TrainerID",
    "JockeyID",
    "NoFrontCover",
    "PositionInRunning",
    "WideOffRail",
    "FinishedFirstSeven",
    "FinishedBS",
    "FinishedPU",
    "FinishedFL",
    "FinishedNP",
    "FinishedUN",
    "FinishedDQ",
    "FinishedWC",
    "FinishedUR",
    "h_average_finish_position",
    "j_average_finish_position",
    "t_average_finish_position",
    'TotalPrize',
]


clean_df = df.drop(columns=filter_columns)
clean_df.to_pickle("clean_df.pkl")

In [33]:
clean_df.head()

,BeatenMargin,FinishPosition,handicap_distance,horse_age,HorseID,PIRPosition,PriceSP,Prizemoney,RaceID,RaceStartTime,...,saddle_cloth_9,starting_line_-1,starting_line_1,starting_line_2,wetness_scale_1,wetness_scale_3,wetness_scale_4,wetness_scale_7,wetness_scale_9,target
0,0.00,1,0.0,3,1519537,0,2.25,0.0,1582365,2013-12-09 12:30:00,...,0,1,0,0,0,1,0,0,0,0.00
1,0.00,2,0.0,3,1518276,0,8.20,20700.0,1582365,2013-12-09 12:30:00,...,0,1,0,0,0,1,0,0,0,0.45
2,12.70,3,0.0,3,1516767,0,44.00,11500.0,1582365,2013-12-09 12:30:00,...,0,1,0,0,0,1,0,0,0,0.25
3,14.25,4,0.0,3,1516646,0,5.00,6440.0,1582365,2013-12-09 12:30:00,...,0,1,0,0,0,1,0,0,0,0.14
4,25.05,5,0.0,3,1515089,0,41.00,3680.0,1582365,2013-12-09 12:30:00,...,0,1,0,0,0,1,0,0,0,0.08
